In [54]:
import pandas as pd
import numpy as np
import datetime

In [55]:
data = pd.read_csv("vehicles.csv", header = None, skiprows=[0])
column_headers = pd.read_csv("vehicles.csv", nrows=0).columns.tolist()
data.columns = column_headers
data = data.drop(["id", "url", "region_url", "VIN", "size", "image_url", "description", "county", "lat", "long"], axis=1)
data = data.dropna(subset=['posting_date'])
#data = data.dropna(subset=['condition'])
#drop rows where the manufacturer and model are both null (that should also eleminate rows missing the type as well)
#drop rows where both the title_status and condition are null
data = data.dropna(subset=["title_status", "condition"], how='all')
#drop rows where the posting date is null and the odometer is null
data = data.dropna(subset=["year", "odometer"], how='all')
#drop the rows where the posting date is null

#print(data.isna().sum())
#id, url, region_url, VIN, size, image_url, description, county, lat, long
#year undecided
#manufacturer fill null values by mapping the model to who is the maker of the model is - Curtis
#model drop the missing model values for now - Curtis
#condition when: - Cole
# title_status clean and odometer less than 500 set to new, 
# title_status clean and odometer less than 5000 set to like new, 
# title_status clean and odometer less than 40000 is excellent, 
# title_status clean and when the odometer is less than 80000 is good,
# title_status clean and when the odometer is less than 120000 is fair,
# title_status salvage then map to salvage
#cylinders drop the column?
#fuel just fill in as gas - Cole
#odometer fill by taking posting_date year - year x 20000 = fill value - Cole
#title_status fill: - Cole
# new, like new excellent, good and fair put as clean
# salvage as salvage
#transmission put as automatic - Curtis
#Option 1: drive taking count for region and then assigning the type based on what is most common in that region
#Option 2: take count for the vehicle type and then assign the type for what is most common for that type
#type map the model to the type - Curtis
#paint_color assign based on the type and what is the most popular for that vehicle type - Curtis

# null_rows = data[data[['title_status', 'condition']].isnull().all(axis=1)]
# print(len(null_rows))
#print(data.type.value_counts())
#print(data.condition.value_counts())
#print(data.shape[0])
#display(data)


In [ ]:
#Curtis cell for data cleaning

In [56]:
#Cole cell for data cleaning
title_status_mapping = {"new" : "clean", "like new" : "clean", "excellent" : "clean", "good" : "clean", "fair" : "clean", "salvage" : "salvage" }
def format_dates(old_format):
    new_format = old_format.split("-")
    #new_format = datetime.datetime.strptime(old_format,"%Y-%m-%dT%H:%M:%S-%f").date()
    return int(new_format[0])

def fill_conditions(data_row):
# title_status clean and odometer less than 500 set to new, 
    vehicle_condition = data_row["condition"]
    if pd.isna(data_row["condition"]) == False:
        vehicle_condition = data_row["condition"]
    elif (data_row["title_status"] == "clean" or data_row["title_status"] == "lien" or data_row["title_status"] == "missing") and data_row["odometer"] < 500:
        vehicle_condition = "new"
# title_status clean and odometer less than 5000 set to like new, 
    elif (data_row["title_status"] == "clean" or data_row["title_status"] == "lien" or data_row["title_status"] == "missing") and data_row["odometer"] < 5000:
        vehicle_condition = "like new"
# title_status clean and odometer less than 40000 is excellent, 
    elif (data_row["title_status"] == "clean" or data_row["title_status"] == "lien" or data_row["title_status"] == "missing") and data_row["odometer"] < 40000:
        vehicle_condition = "excellent"
# title_status clean and when the odometer is less than 80000 is good,
    elif (data_row["title_status"] == "clean" or data_row["title_status"] == "lien" or data_row["title_status"] == "missing") and data_row["odometer"] < 80000:
        vehicle_condition = "good"
# title_status clean and when the odometer is less than 120000 is fair,
    elif (data_row["title_status"] == "clean" or data_row["title_status"] == "lien" or data_row["title_status"] == "missing") and data_row["odometer"] <= 150000:
        vehicle_condition = "fair"
# title_status salvage then map to salvage
    else:
        vehicle_condition = "salvage"
    return vehicle_condition

data["fuel"] = data["fuel"].fillna(value="gas")
data["title_status"] = data["title_status"].fillna(value=data["condition"].map(title_status_mapping))
data["posting_date"] = data["posting_date"].apply(format_dates)
data["odometer"] = data["odometer"].fillna(value=(data["posting_date"] - data["year"]) * 20000)
data["condition"] = data.apply(fill_conditions, axis=1)
# print(data.isna().sum())
# null_rows = data[data[['condition']].isnull().all(axis=1)]
# print(null_rows.title_status.value_counts())
# print(data.condition.value_counts())
# print(null_rows.odometer.value_counts())

In [37]:
from sklearn.model_selection import train_test_split

y = data["price"]
y.columns = ["price"]
X = data.drop("price", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [38]:
#code for analysis

In [39]:
#code for model

In [40]:
#code for validation